In [1]:
#import python packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import multiprocessing
from multiprocessing.dummy import Pool as ThreadPool 
import time
from functools import partial 
import os
%matplotlib inline

In [2]:
#Define file directory and name
DATA_DIR = "../gamedata/original_dataset/"
DATA_FILE = "dd_event_schemas.csv"

In [3]:
#Load original dataset
dataset=pd.read_csv(DATA_DIR+DATA_FILE)

In [4]:
#Select a sub dataset where the genre equals to 'No genre' 
subdataset = dataset[dataset['genre']=='Poker']
#List of all events in 'No genre' subdataset
events = subdataset['eventname'].drop_duplicates()
#Count the amount of all event
#events_amount=events.count()
#List of all gameid  in 'No genre' subdataset
games = subdataset['gameid'].drop_duplicates()

In [5]:
eventsValueCount = subdataset['eventname'].value_counts()

In [6]:
eventsValueCount.to_csv(DATA_DIR+'eventsVocabulary.csv')

In [7]:
eventsValueCount = pd.read_csv(DATA_DIR+'eventsVocabulary.csv',names=['events','num'])

In [8]:
eventsVocabulary = eventsValueCount['events']

In [9]:
events = list(eventsVocabulary)

In [10]:
#Get all gameid in this genre
def get_all_gameid(dataset):
    AllGameId = dataset['gameid']
    AllGameId = AllGameId.drop_duplicates()
    
    return AllGameId

In [11]:
#Get all events in this game
def get_all_gameEvents(gameId,dataset):
    findGame = dataset[dataset['gameid'] == gameId]
    eventsInGame = findGame.drop_duplicates(['eventname'])
    eventsList = eventsInGame['eventname']
    
    return eventsList

In [12]:
#Obtain all game id
allGameId = get_all_gameid(subdataset)
len(events)

271

In [13]:
#Extract features
def get_feature_vector(gameId,eventsVocabulary,dataset):
    ''' Map the presence of events to a vector
    :param gameId
    :type gameId: list of int
    :param eventsVocabulary 
    :type eventsVocabulary : list of str
    :param dataset
    :type dataset: dataframe
    :return feature_vect
    :type feature_vect: list of int
    '''
    eventsList = get_all_gameEvents(gameId,dataset)
    feature_vect = [0]*len(eventsVocabulary)
    
    for event in eventsList:
        if event in eventsVocabulary:
            idx = eventsVocabulary.index(event)
            print(idx)
            feature_vect[idx] = 1
    return feature_vect      

In [ ]:
#Get all feature vectors
train_dataset = [get_feature_vector(gameId,events,subdataset) for gameId in allGameId]

In [15]:
train_dataset = pd.DataFrame(train_dataset)
type(train_dataset)

pandas.core.frame.DataFrame

In [16]:
train_dataset_tmp = np.array(train_dataset)

In [17]:
#amount of each event
num_events = np.sum(train_dataset,axis=0)

In [18]:
#Calculate P(ei|ej)
def calculateProb(i):
    eiej_tmp = pd.DataFrame()
    arr1=[]
    arr2=[]
    #amount = num 
    #for i in amount:
    arr1=[]
    for j in range(len(events)):
        if i==j:
            p = 0.0
        else:
            eiej_tmp['i'] = train_dataset.iloc[:,i]
            eiej_tmp['j'] = train_dataset.iloc[:,j]
            eiej = eiej_tmp[(eiej_tmp['i']==1) & (eiej_tmp['j']==1)]
            p = len(eiej)/np.sum(eiej_tmp['j'])
            #print('p(e'+str(j)+'|e'+str(i)+ ')='+str(p) )
        arr1.append(p)
    print(arr1)
    #arr2.append(arr1)
    #print(i)
    return arr1

In [ ]:
start = time.clock()
amount = range(len(events))
pool = ThreadPool(32) 
matrix = pool.map(calculateProb, amount)
pool.close()
pool.join()
end = time.clock()

In [30]:
#Time to create the probability matrix
print(end-start)
newmatrix = pd.DataFrame(matrix)

148.692996


In [130]:
#Change input game by deleting or adding '#' below
#input_events = get_all_gameEvents(59,subdataset)
input_events = get_all_gameEvents(3517,subdataset)

In [131]:
#Write the matrix to a csv file
newmatrix.to_csv(DATA_DIR+'probMatrix.csv',index=False)

In [19]:
#Recommend events
def recommend(input_events):
    probMatrix = pd.read_csv(DATA_DIR + 'probMatrix.csv')
    new_feature=['NaN']*len(events)
    predict_list =[]
    for data in input_events:
        if data in events:
            idx = events.index(data)
            new_feature[idx]=data
            #print(data,idx)
            find = probMatrix.iloc[idx]
            #print(find)
            findValue = find.idxmax()
            findValue = int(findValue)
            #find[findValue] = 0.0
           # print(find[findValue])
            if find[findValue]!=0:
                new_feature[findValue]=events[findValue]
                find[findValue]=0.0
                if events[findValue] not in predict_list:
                    predict_list.append(events[findValue])
            findValue2 = find.idxmax()
            findValue2 = int(findValue2)
            if find[findValue2]!=0:
                new_feature[findValue2]=events[findValue2]
                find[findValue2]=0.0
                if events[findValue2] not in predict_list:
                    predict_list.append(events[findValue2])
            findValue3 = find.idxmax()
            findValue3 = int(findValue3)
            if find[findValue3]!=0:
                new_feature[findValue3]=events[findValue3]
                find[findValue3]=0.0
                if events[findValue3] not in predict_list:
                    predict_list.append(events[findValue3])
            print(data,idx,'--',events[findValue],findValue,'--',events[findValue2],findValue2,'--',events[findValue3],findValue3)
    return predict_list

In [133]:
#Recommend list 1
predict_list1

['playGame',
 'revengePopupEvent',
 'itemBought',
 'transaction',
 'levelUp',
 'appsFlyerAttribution',
 'itemCollected',
 'outOfGameSend',
 'uiInteraction',
 'achievement',
 'clientDevice']

In [134]:
print('number of input events:',len(input_events))
print('number of recommend list:',len(predict_list1))

number of input events: 70
number of recommend list: 11


In [126]:
#Events of game 59 after deleting 'gameStarted' 
events_after_delete1 = {'handSummary', 'achievement', 'missionCompleted',
       'missionContinued', 'tableSearch', 'missionStarted',
       'tableConnect', 'slotReward', 'rewardedItem',
       'messageSent', 'rewardBundle', 'transaction', 'markerEvent',
       'missionAbandoned', 'giftSent', 'videoPokerGameEnd', 'handAction',
       'slotStart', 'videoPokerGameStart', 'rewardedMetric',
       'missionFailed'}

In [127]:
predict_list2 = recommend(events_after_delete1)

rewardedItem 151 -- missionCompleted 91 -- missionContinued 121 -- missionAbandoned 122
achievement 4 -- playGame 7 -- revengePopupEvent 9 -- itemBought 12
missionContinued 121 -- missionCompleted 91 -- missionAbandoned 122 -- slotStart 123
tableConnect 139 -- missionCompleted 91 -- missionContinued 121 -- missionAbandoned 122
videoPokerGameStart 145 -- missionCompleted 91 -- missionContinued 121 -- missionAbandoned 122
handAction 73 -- handSummary 10 -- giftSent 41 -- sendGridEmail 51
handSummary 10 -- giftSent 41 -- sendGridEmail 51 -- giftReceived 64
missionCompleted 91 -- missionContinued 121 -- missionAbandoned 122 -- slotStart 123
tableSearch 154 -- missionCompleted 91 -- missionContinued 121 -- missionAbandoned 122
messageSent 137 -- missionCompleted 91 -- missionContinued 121 -- missionAbandoned 122
missionStarted 42 -- playGame 7 -- revengePopupEvent 9 -- itemBought 12
rewardedMetric 136 -- missionCompleted 91 -- missionContinued 121 -- missionAbandoned 122
giftSent 41 -- hand

In [128]:
predict_list2

['missionCompleted',
 'missionContinued',
 'missionAbandoned',
 'playGame',
 'revengePopupEvent',
 'itemBought',
 'slotStart',
 'handSummary',
 'giftSent',
 'sendGridEmail',
 'giftReceived',
 'levelUp',
 'appsFlyerAttribution',
 'achievement']

In [129]:
print('number of input events:',len(input_events))
print('number of recommend list:',len(predict_list2))

number of input events: 22
number of recommend list: 14


In [117]:
#Events of game 3517 after deleting 'gameStarted' 
events_after_delete2 = {'tournamentPlayAgain', 'playerCreatedTournament',
       'itemBought', 'piggyBank', 'levelUp', 'achievement',
       'blackjackHandResult', 'blackjackAction', 'revengePopupEvent',
       'leaveTeam', 'bonusSpin', 'inviteFriends', 'showScreen',
       'clientDevice', 'updateEvent', 'transaction', 'featureUnlock',
       'teamInvite', 'playerTournamentFinished', 'promoMessageEvent',
       'unlockSaloon', 'uiInteraction', 'playerFriendJoinedTournament',
       'teamChallengeEnded', 'referralClaimReward', 'playGame',
       'tutorial', 'leaveGame', 'economyBalance', 'connectionHandeling',
       'badgeWon', 'loading', 'playerTournamentStarted', 'gifting',
       'doRebuy', 'missionClaimed', 'playerLeftTournament',
       'appsFlyerAttribution', 'gameEnded', 'newPlayer',
       'applicationAction', 'navigationEvent', 'missionStarted',
       'chatEvent', 'privateGameAction', 'shareImage', 'missionPaused',
       'claimCouponEvent', 'engageResponse', 'trophyWon', 'options',
       'joinTeam', 'notificationServices', 'outOfGameSend',
       'itemCollected', 'offerEvent', 'challengeStartEvent',
       'challengeRewardClaimedEvent', 'mutePlayer', 'reportPlayer',
       'notificationOpened', 'showNormalAd', 'referralAction',
       'teamChallengeStarted', 'teamChallengeRewardClaimed', 'rateGame',
       'teamRankChange', 'teamKick', 'createTeam'}

In [119]:
predict_list3 = recommend(events_after_delete2)

featureUnlock 37 -- playGame 7 -- revengePopupEvent 9 -- itemBought 12
unlockSaloon 47 -- playGame 7 -- revengePopupEvent 9 -- itemBought 12
achievement 4 -- playGame 7 -- revengePopupEvent 9 -- itemBought 12
teamInvite 18 -- playGame 7 -- revengePopupEvent 9 -- itemBought 12
claimCouponEvent 61 -- playGame 7 -- revengePopupEvent 9 -- itemBought 12
piggyBank 48 -- playGame 7 -- revengePopupEvent 9 -- itemBought 12
gifting 40 -- playGame 7 -- revengePopupEvent 9 -- itemBought 12
referralAction 63 -- playGame 7 -- revengePopupEvent 9 -- itemBought 12
updateEvent 81 -- playGame 7 -- revengePopupEvent 9 -- itemBought 12
doRebuy 24 -- playGame 7 -- revengePopupEvent 9 -- itemBought 12
notificationServices 86 -- levelUp 1 -- outOfGameSend 5 -- playGame 7
promoMessageEvent 31 -- playGame 7 -- revengePopupEvent 9 -- itemBought 12
newPlayer 11 -- levelUp 1 -- appsFlyerAttribution 2 -- clientDevice 3
itemCollected 14 -- playGame 7 -- revengePopupEvent 9 -- itemBought 12
teamRankChange 23 -- play

In [102]:
predict_list3

['playGame',
 'revengePopupEvent',
 'itemBought',
 'levelUp',
 'outOfGameSend',
 'appsFlyerAttribution',
 'clientDevice',
 'uiInteraction',
 'itemCollected',
 'achievement']

In [106]:
print('number of input events:',len(input_events))
print('number of recommend list:',len(predict_list3))

number of input events: 70
number of recommend list: 10
